In [29]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\praka\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [30]:
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [31]:
X_train_text, X_test_text, y_train, y_test, vectorizer = pd.read_pickle("../datasets/processed_data.pkl")

In [32]:
log_model = LogisticRegression(max_iter=500)
log_model.fit(X_train_text, y_train)
log_y_pred = log_model.predict(X_test_text)
log_accuracy = accuracy_score(y_test, log_y_pred)
print(f"Logistic Regression Accuracy: {log_accuracy * 100:.2f}%")
print("Logistic Regression Report:\n", classification_report(y_test, log_y_pred))

Logistic Regression Accuracy: 89.70%
Logistic Regression Report:
               precision    recall  f1-score   support

           0       0.89      0.90      0.90      4071
           1       0.90      0.89      0.90      4016

    accuracy                           0.90      8087
   macro avg       0.90      0.90      0.90      8087
weighted avg       0.90      0.90      0.90      8087



In [33]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_text, y_train)
rf_y_pred = rf_model.predict(X_test_text)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print(f"Random Forest Accuracy: {rf_accuracy * 100:.2f}%")
print("Random Forest Report:\n", classification_report(y_test, rf_y_pred))

Random Forest Accuracy: 88.35%
Random Forest Report:
               precision    recall  f1-score   support

           0       0.89      0.88      0.88      4071
           1       0.88      0.89      0.88      4016

    accuracy                           0.88      8087
   macro avg       0.88      0.88      0.88      8087
weighted avg       0.88      0.88      0.88      8087



In [34]:
nb_model = MultinomialNB()
nb_model.fit(X_train_text, y_train)
nb_y_pred = nb_model.predict(X_test_text)
nb_accuracy = accuracy_score(y_test, nb_y_pred)
print(f"Naïve Bayes Accuracy: {nb_accuracy * 100:.2f}%")
print("Naïve Bayes Report:\n", classification_report(y_test, nb_y_pred))

Naïve Bayes Accuracy: 86.97%
Naïve Bayes Report:
               precision    recall  f1-score   support

           0       0.89      0.84      0.87      4071
           1       0.85      0.90      0.87      4016

    accuracy                           0.87      8087
   macro avg       0.87      0.87      0.87      8087
weighted avg       0.87      0.87      0.87      8087



In [35]:
MAX_NUM_WORDS = 5000
MAX_SEQUENCE_LENGTH = 200

In [37]:
from scipy.sparse import issparse

if issparse(X_train_text):
    X_train_text = X_train_text.toarray()


In [39]:
# Convert sparse matrix to text
X_train_text = vectorizer.inverse_transform(X_train_text)
X_train_text = [" ".join(words) for words in X_train_text]  # Convert list of word arrays into sentences

X_test_text = vectorizer.inverse_transform(X_test_text)
X_test_text = [" ".join(words) for words in X_test_text]  # Same for test data

# Tokenization
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train_text)  

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train_text)  
X_test_seq = tokenizer.texts_to_sequences(X_test_text)  

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH)


In [41]:
lstm_model = Sequential([
    Embedding(MAX_NUM_WORDS, 128, input_length=MAX_SEQUENCE_LENGTH),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [42]:
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
lstm_model.fit(X_train_pad, np.array(y_train), epochs=5, batch_size=32, validation_data=(X_test_pad, np.array(y_test)))

Epoch 1/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 126s 122ms/step - accuracy: 0.8078 - loss: 0.3881 - val_accuracy: 0.8885 - val_loss: 0.2639
Epoch 2/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 141s 140ms/step - accuracy: 0.9225 - loss: 0.1909 - val_accuracy: 0.8924 - val_loss: 0.2567
Epoch 3/5
 537/1011 ━━━━━━━━━━━━━━━━━━━━ 1:06 140ms/step - accuracy: 0.9393 - loss: 0.1495

In [ ]:
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test_pad, np.array(y_test))
print(f"LSTM Model Accuracy: {lstm_accuracy * 100:.2f}%")

In [ ]:
print(f"Vocabulary Size: {len(tokenizer.word_index)}")
X_test_pad = pad_sequences(X_test_seq, maxlen=X_train_pad.shape[1])

In [ ]:
X_test_pad = vectorizer.transform(X_test_text)

In [ ]:
accuracy = model.score(X_test_pad, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_pad)
print(classification_report(y_test, y_pred))


In [ ]:
import pandas as pd

X_test_text_reset = pd.Series(X_test_text).reset_index(drop=True)


In [ ]:
for idx in misclassified_idx[:5]:  
    print(f"Actual: {y_test_reset[idx]}, Predicted: {y_pred[idx]}")  
    print(f"Review: {X_test_text[idx]}")  # No need for .reset_index()
    print("-" * 80)

In [ ]:
#training hybrid model from weak model

In [ ]:
best_model = max(
    [(log_model, log_accuracy), (rf_model, rf_accuracy), (nb_model, nb_accuracy)], key=lambda x: x[1]
)[0]

In [ ]:
if lstm_accuracy > max(log_accuracy, rf_accuracy, nb_accuracy):
    lstm_model.save("../models/lstm_model.h5")
    best_model = "LSTM Model"
else:
    with open("../models/model.pkl", "wb") as model_file:
        pickle.dump(best_model, model_file)
    with open("../models/vectorizer.pkl", "wb") as vectorizer_file:
        pickle.dump(vectorizer, vectorizer_file)

In [ ]:
print(f"Best model selected: {best_model}")
print("Model training complete.")